In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix, precision_recall_curve, auc

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from imblearn.over_sampling import SMOTE
import joblib

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

print("Librerías cargadas correctamente.")


Librerías cargadas correctamente.


In [2]:

def cargar_datasets(ruta_base: str) -> dict:
    archivos = {
        'customers': 'olist_customers_dataset.csv',
        'geolocation': 'olist_geolocation_dataset.csv',
        'items': 'olist_order_items_dataset.csv',
        'payments': 'olist_order_payments_dataset.csv',
        'reviews': 'olist_order_reviews_dataset.csv',
        'orders': 'olist_orders_dataset.csv',
        'products': 'olist_products_dataset.csv',
        'sellers': 'olist_sellers_dataset.csv',
        'categories': 'product_category_name_translation.csv'
    }
    
    data = {}
    for key, archivo in archivos.items():
        try:
            df = pd.read_csv(f"{ruta_base}/{archivo}")
            data[key] = df
            print(f"{key} cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
            if df.isnull().sum().sum() > 0:
                print(f" Contiene valores nulos: {df.isnull().sum().sum()}")
        except FileNotFoundError:
            print(f"Archivo no encontrado: {archivo}")
        
    return data

# Llamada a la función
data = cargar_datasets("../data/raw")



customers cargado: 99441 filas, 5 columnas
geolocation cargado: 1000163 filas, 5 columnas
items cargado: 112650 filas, 7 columnas
payments cargado: 103886 filas, 5 columnas
reviews cargado: 99224 filas, 7 columnas
 Contiene valores nulos: 145903
orders cargado: 99441 filas, 8 columnas
 Contiene valores nulos: 4908
products cargado: 32951 filas, 9 columnas
 Contiene valores nulos: 2448
sellers cargado: 3095 filas, 4 columnas
categories cargado: 71 filas, 2 columnas


In [3]:

orders = data['orders'][['customer_id', 'order_id', 'order_purchase_timestamp']].copy()
reviews = data['reviews'][['review_id', 'order_id', 'review_creation_date']].copy()

# Convertir fechas a tipo datetime
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])

print("Datasets base preparados correctamente.")


Datasets base preparados correctamente.


In [8]:

# Número total de pedidos por cliente
df_orders = (
    orders.groupby('customer_id')
    .agg(
        total_orders=('order_id', 'count'),
        last_purchase=('order_purchase_timestamp', 'max'),
        first_purchase=('order_purchase_timestamp', 'min')
    )
    .reset_index()
)

# Calcular promedio de días entre compras (frecuencia)
df_orders['avg_days_between_orders'] = (
    (df_orders['last_purchase'] - df_orders['first_purchase']).dt.days /
    (df_orders['total_orders'] - 1)
).replace([np.inf, np.nan], 0)

print("Datos agregados por cliente a partir de órdenes.")
df_orders.head()
df_orders['total_orders'].value_counts().sort_index()


Datos agregados por cliente a partir de órdenes.


total_orders
1    99441
Name: count, dtype: int64

In [73]:

def generar_features_clientes(df_orders, df_items, df_products):
    """
    Genera features agregadas por cliente sin usar recency (días desde la última compra):
    - Frecuencia y gasto
    - Diversidad de productos y categorías
    - Promedio de reseñas
    - Tiempo promedio y desviación entre pedidos
    """
    # Frecuencia de compra y monetario
    freq_monetary = df_orders.groupby('customer_unique_id').agg({
        'order_id':'nunique',
        'payment_value':['mean','sum'],
        'total_price':'sum',
        'total_freight':'mean'
    })
    freq_monetary.columns = ['n_pedidos','avg_payment_value','total_payment','total_price','avg_freight']
    freq_monetary.reset_index(inplace=True)

    # Diversidad de productos y categorías
    # Merge items con info de producto para obtener categoría
    df_items_full = df_items.merge(
        df_products[['product_id','product_category_name']],
        on='product_id',
        how='left'
    )

    # Merge con customer_id vía orders
    df_items_unique = df_items_full.merge(
        df_orders[['order_id','customer_unique_id']],
        on='order_id',
        how='left'
    )

    diversidad = df_items_unique.groupby('customer_unique_id').agg({
        'product_id':'nunique',
        'product_category_name':'nunique'
    }).rename(columns={
        'product_id':'n_productos_distintos',
        'product_category_name':'n_categorias_distintas'
    }).reset_index()

    # Reviews: promedio de puntuación
    reviews = df_orders.groupby('customer_unique_id').agg({
        'review_score':'mean'
    }).rename(columns={'review_score':'avg_review_score'}).reset_index()

    # Tiempo entre pedidos histórico
    df_sorted = df_orders.sort_values(['customer_unique_id','order_purchase_timestamp'])
    df_sorted['prev_order_date'] = df_sorted.groupby('customer_unique_id')['order_purchase_timestamp'].shift(1)
    df_sorted['dias_entre_pedidos'] = (df_sorted['order_purchase_timestamp'] - df_sorted['prev_order_date']).dt.days

    tiempo_entre = df_sorted.groupby('customer_unique_id')['dias_entre_pedidos'].agg(['mean','std']).reset_index()
    tiempo_entre.rename(columns={'mean':'avg_dias_entre_pedidos','std':'std_dias_entre_pedidos'}, inplace=True)

    # Merge de todas las features en un solo DataFrame
    features = freq_monetary.merge(diversidad, on='customer_unique_id', how='left')
    features = features.merge(reviews, on='customer_unique_id', how='left')
    features = features.merge(tiempo_entre, on='customer_unique_id', how='left')

    print(f"Features generadas: {features.shape[1]} columnas")
    return features

features_clientes = generar_features_clientes(df, data['items'], data['products'])
features_clientes.head()


Features generadas: 11 columnas


,customer_unique_id,n_pedidos,avg_payment_value,total_payment,total_price,avg_freight,n_productos_distintos,n_categorias_distintas,avg_review_score,avg_dias_entre_pedidos,std_dias_entre_pedidos
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.900,141.900,129.900,12.000,1.000,1.000,5.000,NaN,NaN
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.190,27.190,18.900,8.290,1.000,1.000,4.000,NaN,NaN
2,0000f46a3911fa3c0805444483337064,1,86.220,86.220,69.000,17.220,1.000,1.000,3.000,NaN,NaN
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.620,43.620,25.990,17.630,1.000,1.000,4.000,NaN,NaN
4,0004aac84e0df4da2b147fca70cf8255,1,196.890,196.890,180.000,16.890,1.000,1.000,5.000,NaN,NaN


In [80]:
def preparar_datos(features: pd.DataFrame, labels: pd.DataFrame, test_size: float = 0.2, random_state: int = 42):

    # Merge features y etiquetas
    df_modelo = features.merge(labels, on='customer_unique_id', how='inner')
    df_modelo.dropna(subset=['churn'], inplace=True)

    # Separar X e y
    X = df_modelo.drop(columns=['customer_unique_id','churn'])
    y = df_modelo['churn']

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=test_size, random_state=random_state
    )

    print(f"Train: {X_train.shape}, Test: {X_test.shape}")

    # Ver distribución de clases en entrenamiento
    prop_train = y_train.value_counts(normalize=True)
    print("Distribución de clases en entrenamiento:\n", prop_train)

    # Aplicar SMOTE si la clase minoritaria < 20%
    if prop_train.min() < 0.2:
        print("⚡ Clase minoritaria <20%, aplicando SMOTE para balancear...")
        
        X_train = X_train.select_dtypes(include=['int64','float64','Int64','Float64'])
        X_test = X_test.select_dtypes(include=['int64','float64','Int64','Float64'])
        smote = SMOTE(random_state=random_state)
        X_train, y_train = smote.fit_resample(X_train, y_train)
        print("Distribución de clases después de SMOTE:")
        print(pd.Series(y_train).value_counts(normalize=True))

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = preparar_datos(features_clientes, clientes_churn)

Train: (23361, 11), Test: (5841, 11)
Distribución de clases en entrenamiento:
 churn
1   0.978
0   0.022
Name: proportion, dtype: float64
⚡ Clase minoritaria <20%, aplicando SMOTE para balancear...


ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [41]:
# Crear imputador que reemplaza NaN por la media de la columna
imputer = SimpleImputer(strategy='mean')

# Ajustar e imputar en X_train y X_test
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

In [39]:
# Detectar columnas datetime
datetime_cols = X_train.select_dtypes(include=['datetime64[ns]']).columns
print("Columnas datetime que se eliminarán:", list(datetime_cols))

# Eliminar columnas datetime de X_train y X_test
X_train = X_train.drop(columns=datetime_cols)
X_test = X_test.drop(columns=datetime_cols)


Columnas datetime que se eliminarán: []


In [42]:

def entrenar_y_evaluar_modelos(X_train, X_test, y_train, y_test):

    modelos = {
        "LogisticRegression": LogisticRegression(max_iter=1000, class_weight='balanced'),
        "RandomForest": RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
        "GradientBoosting": HistGradientBoostingClassifier(random_state=42),
        "KNN": KNeighborsClassifier(),
        "NaiveBayes": GaussianNB(),
        "SVM": SVC(probability=True, class_weight='balanced')
    }

    resultados = []
    for nombre, modelo in modelos.items():
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        y_prob = modelo.predict_proba(X_test)[:,1] if hasattr(modelo, "predict_proba") else None
        
        roc = roc_auc_score(y_test, y_prob) if y_prob is not None else np.nan
        f1 = f1_score(y_test, y_pred)

        resultados.append({'modelo': nombre, 'roc_auc': roc, 'f1': f1})
        print(f"{nombre}: ROC-AUC={roc:.3f}, F1={f1:.3f}")

    return pd.DataFrame(resultados).sort_values(by='roc_auc', ascending=False)

resultados_modelos = entrenar_y_evaluar_modelos(X_train, X_test, y_train, y_test)
resultados_modelos


c:\Users\ASUS\olist-data\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression: ROC-AUC=1.000, F1=1.000
RandomForest: ROC-AUC=1.000, F1=1.000
GradientBoosting: ROC-AUC=0.585, F1=1.000
KNN: ROC-AUC=0.833, F1=1.000
NaiveBayes: ROC-AUC=0.998, F1=0.997
SVM: ROC-AUC=1.000, F1=0.998


,modelo,roc_auc,f1
1,RandomForest,1.000,1.000
0,LogisticRegression,1.000,1.000
5,SVM,1.000,0.998
4,NaiveBayes,0.998,0.997
3,KNN,0.833,1.000
2,GradientBoosting,0.585,1.000


In [43]:

def graficar_metricas(y_test, y_prob, titulo="Curva ROC y Precision-Recall"):
    """
    Genera las curvas ROC y Precision-Recall para evaluar desempeño del modelo.
    """
    from sklearn.metrics import roc_curve, precision_recall_curve, auc

    fpr, tpr, _ = roc_curve(y_test, y_prob)
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    auc_roc = auc(fpr, tpr)
    auc_pr = auc(recall, precision)

    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(fpr, tpr, label=f'ROC AUC={auc_roc:.2f}')
    plt.plot([0,1],[0,1],'--')
    plt.title("Curva ROC")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(recall, precision, label=f'PR AUC={auc_pr:.2f}')
    plt.title("Curva Precision-Recall")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()

    plt.suptitle(titulo)
    plt.show()
